# Imports

In [1]:
#basics
import requests
from bs4 import BeautifulSoup

In [2]:
#some very minimal regex
import re
import regex as regex

In [3]:
#easy-on-the-eye printing (testing purposes)
from pprint import pprint

In [4]:
#writing csvs
import csv
import pandas as pd
import numpy as np

In [5]:
import datetime

In [6]:
#for fake data
%pip install faker
from faker import Faker

Note: you may need to restart the kernel to use updated packages.


In [7]:
#probability of user traveling
from random import seed
from random import randint
from random import choice
from random import uniform

In [8]:
#for the bonus
%pip install selenium
from selenium import webdriver

Note: you may need to restart the kernel to use updated packages.


# Helper Functions

In [9]:
def removeUnrecognizedCountries(countriesDict):
    unrecognizedCountries = ["Abkhazia", "Republic of Artsakh", "Cook Islands", "Kosovo", "Niue", 
                            "Northern Cyprus", "Sahrawi Arab Democratic Republic", "Somaliland",
                            "South Ossetia", "Taiwan", "Transnistria"]
    for uc in unrecognizedCountries:
        countriesDict.pop(uc, None)

# Initialization

In [10]:
seed()

In [11]:
response = requests.get(url="https://en.wikipedia.org/wiki/List_of_sovereign_states")
soup = BeautifulSoup(response.content, features="html.parser")

#get table rows
rows = soup.select_one("table.wikitable.sortable").tbody.children

#get all countries names
countriesDict = {}
for tr in rows:
    if tr.name is not None:
        if tr.td is not None:
            if tr.td.b is not None:
                countryName = tr.td.b.a.get('title')
                if countryName == "Kingdom of the Netherlands":
                    countryName = "Netherlands"
                if countryName == "Danish Realm":
                    countryName = "Denmark"
                countriesDict[countryName] = {"name" : countryName}
                #it may seem redundant now to store the name twice,
                #once as a key
                #and once as a value inside the sub-dictionary
                #however the first makes it easy to access the country to store more data about it
                #and the latter makes it easier to write into csv at the end

In [12]:
removeUnrecognizedCountries(countriesDict)

In [13]:
# pprint(countriesDict)

In [14]:
#for all countries,
#create country url in the form https://en.wikipedia.org/wiki/Country_Name
for country in countriesDict:
    countriesDict[country]["url"] = "https://en.wikipedia.org/wiki/" + countriesDict[country]["name"].replace(" ", "_")

In [15]:
# pprint(countriesDict)

# Global Lists

## Population & Continent

In [16]:
#for all countries,
#get Population and Continent
response = requests.get(url="https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population")
soup = BeautifulSoup(response.content, features="html.parser")
rows = soup.select_one("table.wikitable.sortable").tbody.children

tempDict = {}
for tr in rows:
    if tr.name is not None:
        cells = tr.select("td")
        if cells:
            if cells[0].a is not None:
                cName = cells[0].a.get('title').replace('Demographics of ', '')
                cName = cName.replace('Population of ', '') #special case for Canada
                if cName == "the State of Palestine": #special case for Palestine
                    cName = "State of Palestine"
                if cName in countriesDict.keys():
                    tempDict["continent"] = cells[1].a.string
                    tempDict["population"] = int(cells[2].string.replace(",",""))
                    countriesDict[cName].update(tempDict)
                #else unrecognized country that we are not keeping data for

In [17]:
# pprint(countriesDict)

## Area & Water Percentage

In [18]:
#for all countries,
#get Area and Water Percentage
response = requests.get(url="https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area")
soup = BeautifulSoup(response.content, features="html.parser")
rows = soup.select_one("table.wikitable.sortable").tbody.children


norwayCount = 0 #Norway has multiple entries so we only want the first (largest) one
franceCount = 0 #France has multiple entries so we only want the first (largest) one
tempDict = {}
for tr in rows:
    if tr.name is not None:
        # print (tr)
        cells = tr.select("td")
        # print (cells)
        if cells:
            # print (cells)
            if cells[0].a is not None: #special case because USA has a weird entry that's different from all other countries
                cName = cells[0].a.get('title')
                if cName in countriesDict.keys():
                    areaTemp = cells[1].text
                    areaTempSplit = areaTemp.split("–", 1)
                    areaWithBrackets = areaTempSplit[1]
                    areaSplit = areaWithBrackets.split("(", 1)
                    area = areaSplit[0].replace(",", "")
                    tempDict["area"] = float(area)

                    waterpercentageTemp = cells[4].text
                    waterpercentageSplit = waterpercentageTemp.split("–")
                    waterpercentage = waterpercentageSplit[0]
                    tempDict["water_percentage"] = float(waterpercentage)

                    countriesDict[cName].update(tempDict)

            if cells[1].a is not None:
                cName = cells[1].a.get('title')

                if cName == "Norway":
                    if norwayCount == 1: #skip reappearances of Norway
                        continue
                    else:
                        norwayCount+=1
                    #else we already counted the real Norway previously
                if cName == "France":
                    if franceCount == 1: #skip reappearances of Norway
                        continue
                    else:
                        franceCount+=1
                    #else we already counted the real Norway previously

                if cName in countriesDict.keys():
                    areaWithBrackets = cells[2].text
                    areaSplit = areaWithBrackets.split("(", 1)
                    area = areaSplit[0].replace(",", "")
                    tempDict["area"] = float(area)

                    waterpercentage = cells[5].text
                    if waterpercentage == "Negligible":
                        waterpercentage = 0
                    if cName == "Sudan" or cName == "South Sudan":
                        waterpercentage = waterpercentage.replace("[Note 11]", "")
                    tempDict["water_percentage"] = float(waterpercentage)

                    countriesDict[cName].update(tempDict)
                # else unrecognized country that we are not keeping data for

In [19]:
# pprint (countriesDict)

## GDP (Nominal & PP)

In [20]:
#for all countries,
#get GDP Nominal
response = requests.get(url="https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)")
soup = BeautifulSoup(response.content, features="html.parser")
rows = soup.select_one("table.wikitable.sortable").tbody.children

tempDict = {}
for tr in rows:
    if tr.name is not None:
        cells = tr.select("td")
        if cells:
            if cells[0].a is not None:
                cName = cells[0].a.get('title').replace('Economy of the ', '').replace('Economy of ', '')
                cName = cName.replace("GDP of the ", "").replace("GDP of ", "")
                if cName in countriesDict.keys():
                    if cells[2].get("colspan") == "2":
                        tempDict["gdp_nominal"] = int(cells[3].text.replace(",", ""))
                    else:
                        tempDict["gdp_nominal"] = int(cells[4].text.replace(",", ""))
                    countriesDict[cName].update(tempDict)
#                 #else unrecognized country that we are not keeping data for

#some countries do not appear in the list at all (eg: Vatican City), so we set their GDP Nominal to NULL
for cName, country in countriesDict.items():
    if "gdp_nominal" not in country.keys():
        countriesDict[cName]["gdp_nominal"] = None

In [21]:
# pprint (countriesDict)

In [22]:
#for all countries,
#get GDP PP
response = requests.get(url="https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(PPP)")
soup = BeautifulSoup(response.content, features="html.parser")
rows = soup.select_one("table.wikitable.sortable").tbody.children

tempDict = {}
for tr in rows:
    if tr.name is not None:
        cells = tr.select("td")
        if cells:
            if cells[0].a is not None:
                cName = cells[0].a.get('title').replace('Economy of the ', '').replace('Economy of ', '')
                cName = cName.replace("GDP of the ", "").replace("GDP of ", "")
                if cName in countriesDict.keys():
                    if cells[3].get("colspan") == "2":
                        tempDict["gdp_pp"] = None
                    else:
                        tempDict["gdp_pp"] = int(cells[3].text.replace(",", ""))
                    countriesDict[cName].update(tempDict)
                #else unrecognized country that we are not keeping data for
                
#some countries do not appear in the list at all (eg: Vatican City), so we set their GDP PP to NULL
for cName, country in countriesDict.items():
    if "gdp_pp" not in country.keys():
        countriesDict[cName]["gdp_pp"] = None

In [23]:
pprint (countriesDict)

{'Afghanistan': {'area': 652864.0,
                 'continent': 'Asia',
                 'gdp_nominal': 17876,
                 'gdp_pp': 83370,
                 'name': 'Afghanistan',
                 'population': 32890171,
                 'url': 'https://en.wikipedia.org/wiki/Afghanistan',
                 'water_percentage': 0.0},
 'Albania': {'area': 28748.0,
             'continent': 'Europe',
             'gdp_nominal': 15278,
             'gdp_pp': 43553,
             'name': 'Albania',
             'population': 2829741,
             'url': 'https://en.wikipedia.org/wiki/Albania',
             'water_percentage': 4.7},
 'Algeria': {'area': 2381741.0,
             'continent': 'Africa',
             'gdp_nominal': 171157,
             'gdp_pp': 514748,
             'name': 'Algeria',
             'population': 44700000,
             'url': 'https://en.wikipedia.org/wiki/Algeria',
             'water_percentage': 0.0},
 'Andorra': {'area': 468.0,
             'continent': 'Eur

 'Liberia': {'area': 111369.0,
             'continent': 'Africa',
             'gdp_nominal': 2582,
             'gdp_pp': 7706,
             'name': 'Liberia',
             'population': 4661010,
             'url': 'https://en.wikipedia.org/wiki/Liberia',
             'water_percentage': 13.51},
 'Libya': {'area': 1759540.0,
           'continent': 'Africa',
           'gdp_nominal': 32600,
           'gdp_pp': 92083,
           'name': 'Libya',
           'population': 6959000,
           'url': 'https://en.wikipedia.org/wiki/Libya',
           'water_percentage': 0.0},
 'Liechtenstein': {'area': 160.0,
                   'continent': 'Europe',
                   'gdp_nominal': 6797,
                   'gdp_pp': None,
                   'name': 'Liechtenstein',
                   'population': 39062,
                   'url': 'https://en.wikipedia.org/wiki/Liechtenstein',
                   'water_percentage': 0.0},
 'Lithuania': {'area': 65300.0,
               'continent': 'Europ

                         'water_percentage': 0.0},
 'Tunisia': {'area': 163610.0,
             'continent': 'Africa',
             'gdp_nominal': 38797,
             'gdp_pp': 127325,
             'name': 'Tunisia',
             'population': 11746695,
             'url': 'https://en.wikipedia.org/wiki/Tunisia',
             'water_percentage': 5.04},
 'Turkey': {'area': 783562.0,
            'continent': 'Asia',
            'gdp_nominal': 761425,
            'gdp_pp': 2749570,
            'name': 'Turkey',
            'population': 83614362,
            'url': 'https://en.wikipedia.org/wiki/Turkey',
            'water_percentage': 1.78},
 'Turkmenistan': {'area': 488100.0,
                  'continent': 'Asia',
                  'gdp_nominal': 48276,
                  'gdp_pp': 104539,
                  'name': 'Turkmenistan',
                  'population': 6118000,
                  'url': 'https://en.wikipedia.org/wiki/Turkmenistan',
                  'water_percentage': 3.72},
 'T

## Legislature

In [24]:
#for all countries,
#get Legislature

response = requests.get(url="https://en.wikipedia.org/wiki/List_of_legislatures_by_country")
soup = BeautifulSoup(response.content, features="html.parser")

tableTitle = soup.find(id="Legislatures_of_sovereign_states_(Member_and_observer_states_of_the_United_Nations)")
tbody = (tableTitle.find_next("table")).tbody

tempDict = {}

flags = tbody.select("span.flagicon")

for flag in flags:
    
    countryCell = flag.find_parent("td")
    anchor = countryCell.a
    cName = anchor.get("title")
    
    if cName in countriesDict.keys():
        legisCell = countryCell.nextSibling.nextSibling
        tempString = legisCell.text
        tempStringSplit = tempString.split("[", 1)
        
        legislature = tempStringSplit[0]
        legislature = legislature.replace("\n", "")
        if legislature == "":
            legislature = None
        
        tempDict["legislature"] = legislature
        

    countriesDict[cName].update(tempDict)
        
        
    #else unrecognized country that we are not keeping data for
                    

In [25]:
# pprint (countriesDict)

## Official Languages

In [26]:
#for all countries,
#get Official Languages

response = requests.get(url="https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory")
soup = BeautifulSoup(response.content, features="html.parser")
rows = soup.select_one("table.wikitable").tbody.children

tempDict = {}
for tr in rows:
    if tr.name is not None:
        cells = tr.select("td")
        if cells:
            if cells[0].a is not None:
                cName = cells[0].a.text
                
                if cName == "Palestine":
                    cName = "State of Palestine"
                if cName == "Georgia":
                    cName = "Georgia (country)"
                if cName == "Ireland":
                    cName = "Republic of Ireland"
                if cName == "Gambia":
                    cName = "The Gambia"
                if cName == "Bahamas":
                    cName = "The Bahamas"

                if cName in countriesDict.keys():
                    tempDict["languages"] = []
                    if cells[1].ul is not None:
                        
                        unorderedList = cells[1].ul.find_all("li") #better than children because removes "\n" children
                        
                        for listItem in unorderedList:    
                            tempString = listItem.text
                            tempStringSplit = tempString.split("[", 1)
                            tempString = tempStringSplit[0]
                            tempStringSplit = tempString.split("(", 1)
                            
                            data = tempStringSplit[0]
                            if "None" in data or "No official Language" in data:
                                tempDict["languages"] = None
                            
                            else:
                                tempDict["languages"].append(data)
                            
                    else:
                        tempString = cells[1].text
                        tempStringSplit = tempString.split("[", 1)
                        
                        data = tempStringSplit[0].replace("\n", "")
                        if "None" in data:
                            tempDict["languages"] = None
                        
                        else:
                            tempDict["languages"].append(data)
                        
                        
                    countriesDict[cName].update(tempDict)
                    
                # else unrecognized country that we are not keeping data for

In [27]:
# pprint (countriesDict)

## Time zones

In [28]:
#for all countries,
#get Time zones

response = requests.get(url="https://en.wikipedia.org/wiki/List_of_time_zones_by_country")
soup = BeautifulSoup(response.content, features="html.parser")
rows = soup.select_one("table.wikitable").tbody.children

tempDict = {}
for tr in rows:
    if tr.name is not None:
        cells = tr.select("td")
        if cells:
            if cells[0].a is not None:
                cName = cells[0].a.get("title")
                
                if cName == "Kingdom of the Netherlands":
                    cName = "Netherlands"

                if cName in countriesDict.keys():
                    tempDict["time_zones"] = []
                    
                    if cells[1].text == "1": #if country has only one time zone
                        tzString = cells[2].a.text[3:9]
                            #We want tz to be a float representing the offset from UTC
                            #Floats have decimal points "." not colons ":"
                            #We also want to repalce ± with empty string because ± only comes with UTC 00:00
                            #And 15 minutes = 0.25 hours, 30 minutes = 0.50 hours, and 45 minutes = 0.75 hours
                            #Finally, we replace "−" with "-".
                                #Although very similar in looks, only the second is correctly casted as a negative sign
                        tz = float(tzString.replace(":", ".").replace("±", "").replace("15", "25").replace("30", "50").replace("45", "75").replace("−", "-"))
                        
                        tempDict["time_zones"].append(tz)
                        
                        #else country has multiple time zones
                    else:
                        anchors = cells[2].find_all("a", {"title": re.compile(r'^UTC')})
                        for anchor in anchors:
                            tzString = anchor.text[3:9]
                                #same as above
                            tz = float(tzString.replace(":", ".").replace("±", "").replace("15", "25").replace("30", "50").replace("45", "75").replace("−", "-"))
                        
                            tempDict["time_zones"].append(tz)
                            

                    countriesDict[cName].update(tempDict)
                    
                # else unrecognized country that we are not keeping data for

In [29]:
# pprint (countriesDict)

## Head of State Name + URL

In [30]:
#for all countries,
#get Head of State Name (&URL)
response = requests.get(url="https://en.wikipedia.org/wiki/List_of_current_heads_of_state_and_government")
soup = BeautifulSoup(response.content, features="html.parser")
rows = soup.select_one("table.wikitable").tbody.children

tempDict = {}
for tr in rows:
    if tr.name is not None:
        header = tr.select_one("th")
        if header is not None:
            if header.a is not None:
                if header.a.get("title") != "List of sovereign states":
                    cName = header.a.get("title")
                    
                    if cName == "Kingdom of the Netherlands":
                        cName = "Netherlands"
                    
                    if cName in countriesDict.keys():
                        
                            #Libya and Switzerland have weird table structure
                        if (header.nextSibling) is None:
                            specialAnchor = tr.nextSibling.nextSibling.nextSibling.nextSibling.a
                            tempDict["head_of_state"] = specialAnchor.get("title")
                            tempDict["url_head_of_state"] = "https://en.wikipedia.org" + specialAnchor.get("href")
                            countriesDict[cName].update(tempDict)
                            continue
                        
                        dataCell = header.nextSibling.nextSibling #2 nextSiblings because one of them is "\n"
                        firstAnchor = dataCell.a
                        headOfStateElement = firstAnchor.nextSibling.nextSibling
                        
                        
                            #if the Head of State is placed as text and not as text inside an "a"-tag
                            #(eg: Haiti)
                        if headOfStateElement is None:
                            tempDict["head_of_state"] = firstAnchor.nextSibling.replace(u"\u00A0"+"– ", "")
                            tempDict["url_head_of_state"] =  "https://en.wikipedia.org/wiki/" + tempDict["head_of_state"].replace(" ", "_")
                            
                            #had to hardwire this because of weird structure with [λ] reference tag
                        elif cName == "San Marino":
                            tempDict["head_of_state"] = headOfStateElement.nextSibling.get("title")
                            tempDict["url_head_of_state"] =  "https://en.wikipedia.org" + headOfStateElement.nextSibling.get("href")
                        
                            #Brunei, Oman, Suaid Arabia
                        elif "Prime Minister" in headOfStateElement.string:
                            actualHeadOfStateElement = headOfStateElement.nextSibling.nextSibling #shift by 2 siblings
                            tempDict["head_of_state"] = actualHeadOfStateElement.get("title")
                            tempDict["url_head_of_state"] =  "https://en.wikipedia.org" + actualHeadOfStateElement.get("href")
                        
                            #Remaining countries (most of them)
                        else:
                            tempDict["head_of_state"] = headOfStateElement.get("title")
                            tempDict["url_head_of_state"] =  "https://en.wikipedia.org" + headOfStateElement.get("href")
                        
                        countriesDict[cName].update(tempDict)
                        
##                     else unrecognized country that we are not keeping data for

In [31]:
# pprint (countriesDict)

## Capital City Name + URL

In [32]:
#for all countries,
#get Capital City (& URL)

response = requests.get(url="https://en.wikipedia.org/wiki/List_of_national_capitals")
soup = BeautifulSoup(response.content, features="html.parser")
tbody = soup.select_one("table.wikitable").tbody

tempDict = {}

flags = tbody.select("span.flagicon")

for flag in flags:
    
    countryCell = flag.parent.parent
    anchor = countryCell.a
    cName = anchor.get("title")
    
    if cName in countriesDict.keys():
    
        if countryCell.get("rowspan") == "2" and cName != "State of Palestine":

            currentRow = countryCell.parent
            firstCityCell = currentRow.td

            nextRow = currentRow.nextSibling.nextSibling
            secondCityCell = nextRow.td

            firstCityName = firstCityCell.text
            firstCityURL = "https://en.wikipedia.org" + firstCityCell.a.get("href")

            secondCityName = secondCityCell.text
            secondCityURL = "https://en.wikipedia.org" + secondCityCell.a.get("href")


            #search for official capital city
            if "official" in firstCityName:
                tempStringSplit = firstCityName.split(" (", 1)
                cityName = tempStringSplit[0].strip()
                cityURL = firstCityURL

            elif "official" in secondCityName:
                tempStringSplit = secondCityName.split(" (", 1)
                cityName = tempStringSplit[0].strip()
                cityURL = secondCityURL


            #none is official, search for administrative
            elif "administrative" in firstCityName:
                tempStringSplit = firstCityName.split(" (", 1)
                cityName = tempStringSplit[0].strip()
                cityURL = firstCityURL

            elif "administrative" in secondCityName:
                tempStringSplit = secondCityName.split(" (", 1)
                cityName = tempStringSplit[0].strip()
                cityURL = secondCityURL


            #Only Yemen remains, take 2nd city because Sana'a is more popular
            else:
                tempStringSplit = secondCityName.split(" (", 1)
                cityName = tempStringSplit[0].strip()
                cityURL = secondCityURL


            tempDict["capital_city_name"] = cityName
            tempDict["capital_city_url"] = cityURL

    
        else:
       
            if cName == "Israel":
                tempDict["capital_city_name"] = "Tel Aviv"
                tempDict["capital_city_url"] = "https://en.wikipedia.org/wiki/Tel_Aviv"
            elif cName == "State of Palestine":
                tempDict["capital_city_name"] = "Jerusalem"
                tempDict["capital_city_url"] = "https://en.wikipedia.org/wiki/Jerusalem"
            else:
                cityCell = countryCell.previousSibling.previousSibling #2 previous siblings bec one of them is empty
                cityName = cityCell.a.string
                cityURL = "https://en.wikipedia.org" + cityCell.a.get("href")
                tempDict["capital_city_name"] = cityName
                tempDict["capital_city_url"] = cityURL
            
            
        countriesDict[cName].update(tempDict)
        
        
    # else unrecognized country that we are not keeping data for


                    

In [33]:
# pprint (countriesDict)

## Capital City Population

In [34]:
#for all countries,
#get Capital City Population

#some will be left over, which we will check later
#those that will be left over are the capital cities that are in countries that have multiple capital cities
#(like South Africa has 3)

response = requests.get(url="https://en.wikipedia.org/wiki/List_of_national_capitals_by_population")
soup = BeautifulSoup(response.content, features="html.parser")
tbody = soup.select_one("table.wikitable").tbody

tempDict = {}
for cName, country in countriesDict.items():
    tdAnchors=tbody.select("td a")
    try:
        tempDict={}
        for tdAnchor in tdAnchors:
            cityAnchor = tdAnchor.find(text=country["capital_city_name"])
            if cityAnchor is not None:
                cityCell = cityAnchor.find_parent("td")
                if cityCell is not None:
                    populationCell = cityCell.find_next_sibling("td")
                    populationString = populationCell.text.replace(",", "")

                    if "]" in populationString:
                        tempStringSplit = populationString.split("] ", 1)
                        populationString = tempStringSplit[1]

                    population = int(populationString)
                    tempDict["capital_city_population"] = population
                    break

        #when there are multiple capital cities for a country, I sometimes chose a city that is not in this list
        #These are handled later when iterating over all capital cities
        #This happens in the same cell / code snippet that gets the coordinates of each capital city
        if "capital_city_population" not in tempDict.keys():
            
            population = None
            tempDict["capital_city_population"] = population
        
    except:
        print (cName, ":", country["capital_city_name"], "had an error")
        
    
    countriesDict[cName].update(tempDict)



In [35]:
#City States are countries with only 1 city (their capital city)

#https://en.wikipedia.org/wiki/City-state#Modern_city-states
cityStates = ["Monaco", "Singapore", "Vatican City"]

#City States Should have their capital city population = country population
for cName, country in countriesDict.items():
    if country["capital_city_population"] != country["population"] and cName in cityStates:
        #this only occurs because the the population is accounted for different years
        #so we simply just change the city popualtion (older) to the country population (newer)
        countriesDict[cName]["capital_city_population"] = countriesDict[cName]["population"]

In [36]:
# pprint (countriesDict)

## Capital City Area

In [37]:
#for all countries,
#get Capital City Area

#some will be left over, which we will check later
#those that will be left over are the capital cities that are in countries that have multiple capital cities
#(like South Africa has 3)

response = requests.get(url="https://en.wikipedia.org/wiki/List_of_national_capitals_by_area")
soup = BeautifulSoup(response.content, features="html.parser")
tbody = soup.select_one("table.wikitable").tbody

tempDict = {}
for cName, country in countriesDict.items():
    
    if cName in cityStates:
            countriesDict[cName]["capital_city_area"] = countriesDict[cName]["area"]
            continue
    
    tdAnchors=tbody.select("td a")
    try:
        tempDict={}
        for tdAnchor in tdAnchors:
            
            if "St." in country["capital_city_name"]:
                searchTerm = country["capital_city_name"].replace("St.", "Saint")
            else:
                searchTerm = country["capital_city_name"]
            
            cityAnchor = tdAnchor.find(text=searchTerm)
            
            if cityAnchor is not None:
                cityCell = cityAnchor.find_parent("td")
                if cityCell is not None:
                    areaCell = cityCell.find_next_sibling("td")
                    
                    #countries whose name is the same as the city (but not city States)
                    #these will result in conflict because the country cell is found first
                    #so we find the next TD sibling again
                    if cName == country["capital_city_name"]:
                        areaCell = areaCell.find_next_sibling("td")
                    
                    areaString = areaCell.text.replace(",", "")

                    if "[" in areaString:
                        tempStringSplit = areaString.split("[", 1)
                        areaString = tempStringSplit[0]
                    
                    if "n/a" in areaString:
                        area = None
                    else:
                        area = float(areaString)
                    
                    tempDict["capital_city_area"] = area
                    break
        
        
        
        #Missing values are handled later by accessing the individual capital cities
        if "capital_city_area" not in tempDict.keys():
            area = "CheckLater"
            tempDict["capital_city_area"] = area
        
    except:
        print (cName, ":", country["capital_city_name"], "had an error in Capital City Area")
        
    
    countriesDict[cName].update(tempDict)



In [38]:
# pprint (countriesDict)

# Country-by-Country

## Country: Driving Side, Calling Code, Currency, HDI & GINI

In [39]:
#for all countries,
#get Driving Side, Calling Code, Currency, HDI, GINI
for countryName in countriesDict:

    countryResponse = requests.get( url=countriesDict[countryName]["url"] )

    countrySoup = BeautifulSoup(countryResponse.content, features="html.parser")
    infobox = countrySoup.select_one(".infobox.vcard")

    
    tempDict = {}
    
    #driving side
    try:
        node = infobox.find("a", href="/wiki/Left-_and_right-hand_traffic")
        if node is None:
            data = None
        else:
            row = node.parent.parent
                #we only want one character for the driving side (R or L)
                #and we want that character to be uppercase
            data = row.select_one(".infobox-data").text.upper()[0:1]
        tempDict["driving_side"] = data
    except:
        print(countryName, " had an error in driving side")


    #calling code
    try:
        node = infobox.find("a", text="Calling code")
        if node is None:
            node = infobox.find("th", text="Calling code")
            if node is None:
                data = None
            else:
                row = node.parent
                tempString = row.select_one(".infobox-data").text
                tempStringSplit = tempString.split("[", 1)
                data = tempStringSplit[0]
        else:
            row = node.parent.parent
            tempString = row.select_one(".infobox-data").text
            tempStringSplit = tempString.split("[", 1)
            data = tempStringSplit[0]
        tempDict["calling_code"] = data
    except:
        print(countryName, " had an error in calling code")


    #currency
    try:
        node = infobox.find("th", text="Currency")
        if node is None:
            data = None
        else:
            row = node.parent
            tempString = row.select_one(".infobox-data").text
            tempStringSplit = tempString.split("[", 1)
            data = tempStringSplit[0]
        tempDict["currency"] = data
    except:
        print(countryName, " had an error in currency")


    #HDI
    try:
        node = infobox.find("a", href="/wiki/Human_Development_Index")
        if node is None:
            data = None
        else:
            row = node.parent.parent
                #leading character in string is a no-break space (nbsp) denoted by ASCII value \00A0
                #and we want to replace it with empty char
            infoboxDataTemp = row.select_one(".infobox-data")
            if infoboxDataTemp.sup is None:
                data = float(infoboxDataTemp.img.nextSibling.replace(u"\u00A0", ""))
            else:
                data = float(infoboxDataTemp.sup.previousSibling.replace(u"\u00A0", ""))
        tempDict["hdi"] = data
    except:
        print(countryName, " had an error in HDI")

    #GINI
    try:
        node = infobox.find("a", href="/wiki/Gini_coefficient")
        if node is None:
            tempDict["gini"] = None
        else:
            row = node.parent.parent
                #leading character in string is a no-break space (nbsp) denoted by ASCII value \00A0
                #and we want to replace it with empty char
            if row.select_one(".infobox-data").sup is None:
                data = row.select_one(".infobox-data").br.previousSibling.replace(u"\u00A0", "")
            else:
                data = row.select_one(".infobox-data").sup.previousSibling.replace(u"\u00A0", "")
            tempDict["gini"] = float(data)
    except:
        print(countryName, " had an error in GINI")


    countriesDict[countryName].update(tempDict)
    # print(countryName, " is done.")


In [40]:
# pprint (countriesDict)

## Head of State: Assumed Office, Date of Birth & Political Party

In [41]:
#for all countries,
#get Head of State Date Assumed Office, Date of Birth, Political Party

for countryName in countriesDict:
    
    headOfStateResponse = requests.get( url=countriesDict[countryName]["url_head_of_state"] )

    headOfStateSoup = BeautifulSoup(headOfStateResponse.content, features="html.parser")
    infobox = headOfStateSoup.select_one(".infobox.vcard")
    
    tempDict = {}
    
    #Date Assumed Office
    try:
        node = infobox.find("b", text="Assumed office")
        if node is None:
            node = infobox.find("th", text="Reign")
            if node is None:
                node = infobox.find("th", text="Papacy began") #Vatican City HOS Assumes Office on the same day as the Papacy begans
                
                if node is None:
                    node = infobox.find(text="Since") #Japan
                    
                    if node is None:
                        node = node = infobox.find(text="In reign")
                        
                        if node is None:
                            date = None
                            tempDict["date_assumed_office"] = date
                        
                        else:
                            dateDiv = node.find_next_sibling("div")
                            date = dateDiv.text.replace("–present", "").replace("– present", "").replace("- present", "").replace("– Present", "").strip()
                            tempDict["date_assumed_office"] = date
                    
                    else:
                        dateDiv = node.find_next_sibling("div")
                        date = dateDiv.text
                        tempDict["date_assumed_office"] = date
                
                else:
                        #have to remove " - present" but than can take many forms with different UNICODEs for the - and for the spaces
                        #so I did all replaces that cleaned the data for me
                    dateTemp = node.nextSibling.text.replace("–present", "").replace("– present", "").replace("- present", "").replace("– Present", "").strip()
                    dateTempSplit = dateTemp.split("[", 1)
                    date = dateTempSplit[0].strip()
                    tempDict["date_assumed_office"] = date
            
            else:        
                    #have to remove " - present" but than can take many forms with different UNICODEs for the - and for the spaces
                    #so I did all replaces that cleaned the data for me
                dateTemp = node.nextSibling.text.replace("–present", "").replace("– present", "").replace("- present", "").replace("– Present", "").strip()
                dateTempSplit = dateTemp.split("[", 1) #special case for Thailand
                date = dateTempSplit[0].strip()
                tempDict["date_assumed_office"] = date
        else:
            date = node.parent.nextSibling.nextSibling.nextSibling
            tempDict["date_assumed_office"] = date
    
    except:
        print(countryName, ":", countriesDict[countryName]["head_of_state"], " had an error in Date Assumed Office")
    
    
    
    #Date of Birth
    try:
        node = infobox.find("span", class_="bday") #very clear way of storing bday information
        
        if node is None: #if no clear way exists, we do some meticulous checking
            
            node = infobox.find("th", text="Born")
            if node is None: #no birth information AT ALL is tracked for HOS in infobox
                dateOfBirth = None
                
            else: #some birth information (not full day-month-year)
                birthInformation = node.nextSibling.text
                
                #I learned this regex search technique from this link:
                #https://stackoverflow.com/questions/4510709/find-the-index-of-the-first-digit-in-a-string
                firstDigitIndex = re.search(r"\d", birthInformation)
                
                    #if there are some digits here
                    #I checked them and they all had just the year for the HOS, no month or day
                if firstDigitIndex is not None:
                    dateOfBirth = birthInformation[firstDigitIndex.start():firstDigitIndex.start()+4] #all birth years are 4-digits long
                    
                        #special case for Nauru which has no year but has some digit in the citation [3]
                    if "[" in dateOfBirth or "]" in dateOfBirth:
                        dateOfBirth = None
                    #else dateOfBirth is a valid year, so keep it as it is
                        
                else: #else no dates stored at all, just a location
                    dateOfBirth = None
        
        else: #remaining countries (majority of them)
            dateOfBirth = node.string
        
        tempDict["date_of_birth"] = dateOfBirth
            
    except:
        print(countryName, ":", countriesDict[countryName]["head_of_state"], " had an error in Date of Birth")
    
    
    #Political Party
    dataFlag = False
    try:
        node = infobox.find("th", text="Political party")
        if node is None:
            node = infobox.find("th", text="Political Party")
            if node is None:
                dataFlag = True
                data = None
            else:
                row = node.parent
        else:
            row = node.parent
        
        
        if dataFlag == False:
            tempString = row.select_one(".infobox-data").text
            
                #cleaning the string from citations
            tempStringSplit = tempString.split("[", 1)
            tempString = tempStringSplit[0]
            
                #cleaning the string from start year (it can take multiple forms)
            tempStringSplit = tempString.split("(1", 1) #eg: (1999-present)
            tempString = tempStringSplit[0]
            tempStringSplit = tempString.split("(2", 1) #eg: (2012-present)
            tempString = tempStringSplit[0]
            tempStringSplit = tempString.split("(s", 1) #eg: (since 2005)
            tempString = tempStringSplit[0]
            tempStringSplit = tempString.split("(f", 1) #eg: (from 2007)
            
            data = tempStringSplit[0].strip().replace(u"\u00A0", "")
            
        
        tempDict["political_party"] = data
        
    except:
        print(countryName, ":", countriesDict[countryName]["head_of_state"], " had an error in Political Party")
    
    
    #There are some countries that have some inconsistencies in the Political Party due to poor structure in
    #the pages in Wikipedia.
    #I have noted those countries down and will edit them later.
    
    
    countriesDict[countryName].update(tempDict)
    

In [42]:
# pprint (countriesDict)

## Capital City: Coordinates, Governor & Missing Data

In [43]:
for cName, country in countriesDict.items():
    
    capitalCityResponse = requests.get( url=country["capital_city_url"] )
    capitalCitySoup = BeautifulSoup(capitalCityResponse.content, features="html.parser")
    infobox = capitalCitySoup.select_one(".infobox.vcard")

    tempDict = {}
    
    #Coordinates
    try:
        coordinatesElement = infobox.find("span", class_="geo")
        if coordinatesElement is not None:
            coordinates = coordinatesElement.text.split("; ", 1)
            longitude = coordinates[0]
            latitude = coordinates[1]
            
        else:
            coordinatesElement = capitalCitySoup.find("span", class_="geo")
            if coordinatesElement is not None:
                coordinates = coordinatesElement.text.split("; ", 1) #coordinates may also be found outside of the infobox
                longitude = coordinates[0]
                latitude = coordinates[1]
            
            else: #no coordinates found at all
                longitude = None
                latitude = None
        
        
        tempDict["longitude"] = longitude
        tempDict["latitude"] = latitude
    
    except:
        print (cName, country["capital_city_name"], "had an error in Capital City Coordinates")
    
    
    
    
    #Ruler (Governor / Mayor / Chairman / Etc.)
    try:
        
        synonyms = [
                    "Mayor",
                    "Governor",
                    "President of the Governorate",
                    "Chairman",
                    "Major",
                    "Thrompon",
                    "Maire",
                    "General Manager",
                    "Intendant",
                    "mayor",
                    "Council president"
                    ]


        mayor = None #set default as None. If available, overwrite
        mayorFlag = False
        
        #I learned this regex search technique from this link:
        #https://stackoverflow.com/questions/33406313/how-to-match-any-string-from-a-list-of-strings-in-regular-expressions-in-python
        #I used find_all not find because sometimes I get "Governorate" not "Governor" or "Mayor-Council" not "Mayor"
        #I tried changing my regular expression to look for words terminated by end of line or end of string,,
        #But I was not able to find an answer within the time constraints
        allMatches = infobox.find_all("th", text = regex.compile(r"\L<words>", words=synonyms))

        if not allMatches: #if list is empty
            allMatches = infobox.find_all("a", text = regex.compile(r"\L<words>", words=synonyms))
            if not allMatches:
                mayor = None
                mayorFlag = True
                tempDict["mayor"] = mayor
        
                
        if mayorFlag == False:
            for match in allMatches:
                if "Governorate" not in match.text \
                and "Council" not in match.text \
                and "council" not in match.text \
                and "Strong" not in match.text \
                and "Plaza" not in match.text:
                    parentRow = match.find_parent("tr")
                    if parentRow is not None:
                        nameCell = parentRow.td
                        if nameCell is not None:
                            tempString = nameCell.text
                            tempStringSplit = tempString.split("[", 1)
                            tempString = tempStringSplit[0]
                            tempStringSplit = tempString.split("(", 1)
                            mayor = tempStringSplit[0].strip()
                    break #if a city has a multiple Mayors / Governors / etc., we just want one of them

        tempDict["mayor"] = mayor
    
    except:
        print (cName, country["capital_city_name"], "had an error in Capital City Ruler")
    
    
    
    
    #Missing Population
    if country["capital_city_population"] is None:
        
        tempDict = {}
        
        tempText = infobox.find(text = "Population")
        popTR = tempText.find_parent("tr")
        valueCell = popTR.td
        if valueCell is not None:
            tempString = valueCell.text
            tempStringSplit = tempString.split("[", 1)
            value = int(tempStringSplit[0].strip().replace(",", ""))
        
        else:
            tempString = popTR.find_next_sibling("tr").td.text
            tempStringSplit = tempString.split("[", 1)
            value = int(tempStringSplit[0].strip().replace(",", ""))
        
        tempDict["capital_city_population"] = value
        countriesDict[cName].update(tempDict)


    #Missing Area
    if country["capital_city_area"] == "CheckLater":
        
        tempDict = {}
        
        tempText = infobox.find(text = "Area")
        if tempText is None:
            areaValue = None
        
        else:
            popTR = tempText.find_parent("tr")
            areaValueCell = popTR.td
            
            if areaValueCell is not None:
                tempString = areaValueCell.text
                tempStringSplit = tempString.split("[", 1)
                tempString = tempStringSplit[0]
                tempStringSplit = tempString.split("k", 1)
                areaValue = float(tempStringSplit[0].strip().replace(",", ""))
            
            else:
                tempString = popTR.find_next_sibling("tr").td.text
                tempStringSplit = tempString.split("[", 1)
                tempString = tempStringSplit[0]
                tempStringSplit = tempString.split("k", 1)
                areaValue = float(tempStringSplit[0].strip().replace(",", ""))
        
        
        tempDict["capital_city_area"] = areaValue
        countriesDict[cName].update(tempDict)


    countriesDict[cName].update(tempDict)

In [44]:
pprint(countriesDict)

{'Afghanistan': {'area': 652864.0,
                 'calling_code': '+93',
                 'capital_city_area': 275.0,
                 'capital_city_name': 'Kabul',
                 'capital_city_population': 3140853,
                 'capital_city_url': 'https://en.wikipedia.org/wiki/Kabul',
                 'continent': 'Asia',
                 'currency': 'Afghani (افغانی) (AFN)',
                 'date_assumed_office': '19 August 2021',
                 'date_of_birth': None,
                 'driving_side': 'R',
                 'gdp_nominal': 17876,
                 'gdp_pp': 83370,
                 'gini': None,
                 'hdi': 0.511,
                 'head_of_state': 'Hibatullah Akhundzada',
                 'languages': ['Pashto', 'Dari'],
                 'latitude': '69.17833',
                 'legislature': 'National Assembly (ملی شورا Mili Shura / '
                                'شورای ملی Shura-e Milli)',
                 'longitude': '34.52528',
            

 'Bulgaria': {'area': 111002.0,
              'calling_code': '+359',
              'capital_city_area': 492.0,
              'capital_city_name': 'Sofia',
              'capital_city_population': 1277000,
              'capital_city_url': 'https://en.wikipedia.org/wiki/Sofia',
              'continent': 'Europe',
              'currency': 'Lev (BGN)',
              'date_assumed_office': '22 January 2017',
              'date_of_birth': '1963-06-18',
              'driving_side': 'R',
              'gdp_nominal': 67925,
              'gdp_pp': 174998,
              'gini': 40.0,
              'hdi': 0.816,
              'head_of_state': 'Rumen Radev',
              'languages': ['Bulgarian'],
              'latitude': '23.33',
              'legislature': 'National Assembly (Народно събрание Narodno '
                             'sabranie)',
              'longitude': '42.70',
              'mayor': 'Yordanka Fandakova',
              'name': 'Bulgaria',
              'political_part

 'Equatorial Guinea': {'area': 28051.0,
                       'calling_code': '+240',
                       'capital_city_area': 21.0,
                       'capital_city_name': 'Malabo',
                       'capital_city_population': 100677,
                       'capital_city_url': 'https://en.wikipedia.org/wiki/Malabo',
                       'continent': 'Africa',
                       'currency': 'Central African CFA franc (XAF)',
                       'date_assumed_office': '3 August 1979',
                       'date_of_birth': '1942-06-05',
                       'driving_side': 'R',
                       'gdp_nominal': 11024,
                       'gdp_pp': 26485,
                       'gini': None,
                       'hdi': 0.592,
                       'head_of_state': 'Teodoro Obiang Nguema Mbasogo',
                       'languages': ['French', 'Portuguese', 'Spanish'],
                       'latitude': '8.7737000',
                       'legislature': 

 'Italy': {'area': 301339.0,
           'calling_code': '+39c',
           'capital_city_area': 1285.0,
           'capital_city_name': 'Rome',
           'capital_city_population': 2873104,
           'capital_city_url': 'https://en.wikipedia.org/wiki/Rome',
           'continent': 'Europe',
           'currency': 'Euro (€)b (EUR)',
           'date_assumed_office': '3 February 2015',
           'date_of_birth': '1941-07-23',
           'driving_side': 'R',
           'gdp_nominal': 2003576,
           'gdp_pp': 2610563,
           'gini': 32.8,
           'hdi': 0.892,
           'head_of_state': 'Sergio Mattarella',
           'languages': ['Italian'],
           'latitude': '12.500',
           'legislature': 'Parliament (Parlamento)',
           'longitude': '41.883',
           'mayor': 'Roberto Gualtieri',
           'name': 'Italy',
           'political_party': 'Independent',
           'population': 59108671,
           'time_zones': [1.0],
           'url': 'https://en.wikip

              'water_percentage': 0.0},
 'Mali': {'area': 1240192.0,
          'calling_code': '+223',
          'capital_city_area': 245.0,
          'capital_city_name': 'Bamako',
          'capital_city_population': 1289626,
          'capital_city_url': 'https://en.wikipedia.org/wiki/Bamako',
          'continent': 'Africa',
          'currency': 'West African CFA franc (XOF)',
          'date_assumed_office': '24 May 2021',
          'date_of_birth': '1983',
          'driving_side': 'R',
          'gdp_nominal': 17432,
          'gdp_pp': 49987,
          'gini': 33.0,
          'hdi': 0.434,
          'head_of_state': 'Assimi Goïta',
          'languages': ['French'],
          'latitude': '-8.00278',
          'legislature': 'National Assembly (Assemblée nationale)',
          'longitude': '12.63917',
          'mayor': 'Adama Sangaré',
          'name': 'Mali',
          'political_party': None,
          'population': 20856000,
          'time_zones': [0.0],
          'url': 

          'water_percentage': 0.0},
 'Pakistan': {'area': 907843.0,
              'calling_code': '+92',
              'capital_city_area': 220.0,
              'capital_city_name': 'Islamabad',
              'capital_city_population': 1014825,
              'capital_city_url': 'https://en.wikipedia.org/wiki/Islamabad',
              'continent': 'Asia',
              'currency': 'Pakistani rupee (₨) (PKR)',
              'date_assumed_office': '9 September 2018',
              'date_of_birth': '1949-08-29',
              'driving_side': 'L',
              'gdp_nominal': 263000,
              'gdp_pp': 1110075,
              'gini': 31.6,
              'hdi': 0.557,
              'head_of_state': 'Arif Alvi',
              'languages': ['Urdu', 'English'],
              'latitude': '73.06389',
              'legislature': 'Parliament of Pakistan (پارلیمنٹ)',
              'longitude': '33.69306',
              'mayor': 'Syed Zeeshan Ali Naqvi',
              'name': 'Pakistan',
       

                              'நாடாளுமன்றம் Nāṭāḷumaṉṟam)',
               'longitude': '1.367',
               'mayor': None,
               'name': 'Singapore',
               'political_party': 'Independent',
               'population': 5453600,
               'time_zones': [8.0],
               'url': 'https://en.wikipedia.org/wiki/Singapore',
               'url_head_of_state': 'https://en.wikipedia.org/wiki/Halimah_Yacob',
               'water_percentage': 1.43},
 'Slovakia': {'area': 49037.0,
              'calling_code': '+421',
              'capital_city_area': 367.584,
              'capital_city_name': 'Bratislava',
              'capital_city_population': 424207,
              'capital_city_url': 'https://en.wikipedia.org/wiki/Bratislava',
              'continent': 'Europe',
              'currency': 'Euro (€) (EUR)',
              'date_assumed_office': '15 June 2019',
              'date_of_birth': '1973-06-21',
              'driving_side': 'R',
              'gdp_no

                         'url_head_of_state': 'https://en.wikipedia.org/wiki/Paula-Mae_Weekes',
                         'water_percentage': 0.0},
 'Tunisia': {'area': 163610.0,
             'calling_code': '+216',
             'capital_city_area': 212.63,
             'capital_city_name': 'Tunis',
             'capital_city_population': 767629,
             'capital_city_url': 'https://en.wikipedia.org/wiki/Tunis',
             'continent': 'Africa',
             'currency': 'Tunisian dinar (TND)',
             'date_assumed_office': '23 October 2019',
             'date_of_birth': '1958-02-22',
             'driving_side': 'R',
             'gdp_nominal': 38797,
             'gdp_pp': 127325,
             'gini': 35.8,
             'hdi': 0.74,
             'head_of_state': 'Kais Saied',
             'languages': ['Arabic', 'French'],
             'latitude': '10.18167',
             'legislature': 'Assembly of the Representatives of the People '
                            '(مجلس نو

# Fake Data Generation

In [45]:
fake = Faker()

## Fake User Profiles

In [46]:
userlist = []
for i in range (25):
    
    temp = fake.profile(["username", "sex", "birthdate"])
    
    user = {}
    
    user["username"] = temp["username"]
    user["gender"] = temp["sex"]
    user["dob"] = temp["birthdate"]
    user["email"] = fake.email()
    
    userlist.append(user)


In [47]:
# print (userlist)

## Fake Visits

In [48]:
visits = []
countries = list(countriesDict.keys())
for user in userlist:
    for i in range (1, 10): #user has 10 "opportunities" for a visit
        visit = {}
        if randint(1, 10) <= 7: #70% that each "opportunity" actually becomes an actual visit
            visit["username"] = user["username"]
            visit["country_name"] = choice(countries) #choose a random country
            visit["rating"] =  round(uniform(0,10), 1)
            visit["text_review"] = fake.paragraph(nb_sentences=3)

            date1 = fake.date_object()
            date2 = fake.date_object()
            if date1 < date2: #date1 < date2 means date1 is older
                visit["arrival_date"] = date1
                visit["departure_date"] = date2
            else:
                visit["arrival_date"] = date1
                visit["departure_date"] = date2

            visits.append(visit)

In [49]:
# print (visits)

We should ideally check now that the PK Constraint will hold later on when loading the CSV into the Database, but the odds of this happening with Faker are very low so we can just move along.

# Bonus: Covid Data

## Total Cases

In [50]:
specialURL = "https://en.wikipedia.org/wiki/COVID-19_pandemic_death_rates_by_country"
covResponse = requests.get( url=specialURL )
covSoup = BeautifulSoup(covResponse.content, features="html.parser")
table = covSoup.select_one("table.wikitable.sortable")

rows = table.find_all("tr")

for row in rows:
    currentCell = row.td
    if currentCell is not None:
        anchor = currentCell.a
        if anchor is not None:
            currentName = anchor.text
            
        if currentName == "Palestine":
            currentName = "State of Palestine"
        elif currentName == "Czechia":
            currentName = "Czech Republic"
        elif currentName == "Georgia":
            currentName = "Georgia (country)"
        elif currentName == "Gambia":
            currentName = "The Gambia"
        elif currentName == "Bahamas":
            currentName = "The Bahamas"
        elif currentName == "Timor-Leste":
            currentName = "East Timor"
        elif currentName == "Sao Tome and Principe":
            currentName = "São Tomé and Príncipe"
        elif currentName == "Cabo Verde":
            currentName = "Cape Verde"
        
        if currentName in list(countriesDict.keys()):
            wantedCell = currentCell.find_next_sibling("td").find_next_sibling("td").find_next_sibling("td")
            value = wantedCell.text.strip().replace(",", "")
            countriesDict[currentName]["covid_cases"] = value
                
for cName, country in countriesDict.items():
    if "covid_cases" not in list(country.keys()):
        countriesDict[cName]["covid_cases"] = None
        

In [51]:
# pprint(countriesDict)

## Vaccinations

In [52]:
specialURL = "https://en.wikipedia.org/wiki/Deployment_of_COVID-19_vaccines#cite_note-Template:COVID-19_data-15"
vacResponse = requests.get( url=specialURL )
vacSoup = BeautifulSoup(vacResponse.content, features="html.parser")

table = vacSoup.select_one("table.wikitable.sortable")
rows = table.find_all("tr")
# print (rows)
for row in rows:
    if row.td is not None:
        if row.td.get("colspan") != "4":
            currentCell = row.td.find_next_sibling("td")
            currentName = currentCell.a.text
            
            if currentName == "Palestine":
                currentName = "State of Palestine"
            elif currentName == "Czechia":
                currentName = "Czech Republic"
            elif currentName == "Georgia":
                currentName = "Georgia (country)"
            elif currentName == "Gambia":
                currentName = "The Gambia"
            elif currentName == "Bahamas":
                currentName = "The Bahamas"
            elif currentName == "Timor-Leste":
                currentName = "East Timor"
            elif currentName == "Sao Tome and Principe":
                currentName = "São Tomé and Príncipe"
            elif currentName == "Cabo Verde":
                currentName = "Cape Verde"
            
            if currentName in list(countriesDict.keys()):
                nextCell = currentCell.find_next_sibling("td")
                value = nextCell.text.strip().replace(",", "")
                countriesDict[currentName]["covid_vaccinations"] = value
                
for cName, country in countriesDict.items():
    if "covid_vaccinations" not in list(country.keys()):
        countriesDict[cName]["covid_vaccinations"] = None
        

In [53]:
pprint(countriesDict)

{'Afghanistan': {'area': 652864.0,
                 'calling_code': '+93',
                 'capital_city_area': 275.0,
                 'capital_city_name': 'Kabul',
                 'capital_city_population': 3140853,
                 'capital_city_url': 'https://en.wikipedia.org/wiki/Kabul',
                 'continent': 'Asia',
                 'covid_cases': '156397',
                 'covid_vaccinations': '3398410',
                 'currency': 'Afghani (افغانی) (AFN)',
                 'date_assumed_office': '19 August 2021',
                 'date_of_birth': None,
                 'driving_side': 'R',
                 'gdp_nominal': 17876,
                 'gdp_pp': 83370,
                 'gini': None,
                 'hdi': 0.511,
                 'head_of_state': 'Hibatullah Akhundzada',
                 'languages': ['Pashto', 'Dari'],
                 'latitude': '69.17833',
                 'legislature': 'National Assembly (ملی شورا Mili Shura / '
                      

              'calling_code': '+855',
              'capital_city_area': 679.0,
              'capital_city_name': 'Phnom Penh',
              'capital_city_population': 2011725,
              'capital_city_url': 'https://en.wikipedia.org/wiki/Phnom_Penh',
              'continent': 'Asia',
              'covid_cases': '119092',
              'covid_vaccinations': '13963921',
              'currency': 'Riel (៛) (KHR)',
              'date_assumed_office': '14 October 2004',
              'date_of_birth': '1953-05-14',
              'driving_side': 'R',
              'gdp_nominal': 27097,
              'gdp_pp': 78065,
              'gini': 36.0,
              'hdi': 0.594,
              'head_of_state': 'Norodom Sihamoni',
              'languages': ['Khmer'],
              'latitude': '104.92111',
              'legislature': 'Parliament(សភាតំណាងរាស្ត្រ '
                             'ព្រះរាជាណាចក្រកម្ពុជា)',
              'longitude': '11.56944',
              'mayor': 'Khuong Sreng'

                 'capital_city_url': 'https://en.wikipedia.org/wiki/San_Salvador',
                 'continent': 'Americas',
                 'covid_cases': '116258',
                 'covid_vaccinations': '4341703',
                 'currency': 'United States dollar (USD, since 2001)',
                 'date_assumed_office': '1 June 2019',
                 'date_of_birth': '1981-07-24',
                 'driving_side': 'R',
                 'gdp_nominal': 27022,
                 'gdp_pp': 57952,
                 'gini': 38.8,
                 'hdi': 0.673,
                 'head_of_state': 'Nayib Bukele',
                 'languages': ['Spanish'],
                 'latitude': '-89.19139',
                 'legislature': 'Legislative Assembly (Asamblea Legislativa)',
                 'longitude': '13.69889',
                 'mayor': 'Mario Durán',
                 'name': 'El Salvador',
                 'political_party': 'Nuevas Ideas',
                 'population': 6825935,
       

              'url_head_of_state': 'https://en.wikipedia.org/wiki/Juan_Orlando_Hern%C3%A1ndez',
              'water_percentage': 0.18},
 'Hungary': {'area': 93028.0,
             'calling_code': '+36',
             'capital_city_area': 525.2,
             'capital_city_name': 'Budapest',
             'capital_city_population': 1729040,
             'capital_city_url': 'https://en.wikipedia.org/wiki/Budapest',
             'continent': 'Europe',
             'covid_cases': '892164',
             'covid_vaccinations': '5974514',
             'currency': 'Forint (HUF)',
             'date_assumed_office': '10 May 2012',
             'date_of_birth': '1959-05-09',
             'driving_side': 'R',
             'gdp_nominal': 163469,
             'gdp_pp': 342708,
             'gini': 28.3,
             'hdi': 0.854,
             'head_of_state': 'János Áder',
             'languages': ['Hungarian'],
             'latitude': '19.05139',
             'legislature': 'National Assembly (Orszá

           'capital_city_population': 1184045,
           'capital_city_url': 'https://en.wikipedia.org/wiki/Tripoli',
           'continent': 'Africa',
           'covid_cases': '360914',
           'covid_vaccinations': '1560096',
           'currency': 'Libyan dinar (LYD)',
           'date_assumed_office': '15 March 2021',
           'date_of_birth': '1976-03-03',
           'driving_side': 'R',
           'gdp_nominal': 32600,
           'gdp_pp': 92083,
           'gini': None,
           'hdi': 0.724,
           'head_of_state': 'Mohamed al-Menfi',
           'languages': ['Arabic'],
           'latitude': '13.19139',
           'legislature': 'House of Representatives ( مجلس النواب Majlis '
                          'an-Nuwwab)',
           'longitude': '32.88722',
           'mayor': 'Abdulrauf Beitelmal',
           'name': 'Libya',
           'political_party': None,
           'population': 6959000,
           'time_zones': [2.0],
           'url': 'https://en.wikipedia.org

             'political_party': 'SWAPO',
             'population': 2550226,
             'time_zones': [1.0],
             'url': 'https://en.wikipedia.org/wiki/Namibia',
             'url_head_of_state': 'https://en.wikipedia.org/wiki/Hage_Geingob',
             'water_percentage': 0.12},
 'Nauru': {'area': 21.0,
           'calling_code': '+674',
           'capital_city_area': 1.5,
           'capital_city_name': 'Yaren',
           'capital_city_population': 1100,
           'capital_city_url': 'https://en.wikipedia.org/wiki/Yaren_District',
           'continent': 'Oceania',
           'covid_cases': None,
           'covid_vaccinations': '7612',
           'currency': 'Australian dollar (AUD)',
           'date_assumed_office': '27 August 2019',
           'date_of_birth': None,
           'driving_side': 'L',
           'gdp_nominal': 132,
           'gdp_pp': 132,
           'gini': None,
           'hdi': None,
           'head_of_state': 'Lionel Aingimea',
           'langua

            'capital_city_url': 'https://en.wikipedia.org/wiki/Kigali',
            'continent': 'Africa',
            'covid_cases': '99947',
            'covid_vaccinations': '3937599',
            'currency': 'Rwandan franc (RWF)',
            'date_assumed_office': '22 April 2000',
            'date_of_birth': '1957-10-23',
            'driving_side': 'R',
            'gdp_nominal': 10355,
            'gdp_pp': 31848,
            'gini': 43.7,
            'hdi': 0.543,
            'head_of_state': 'Paul Kagame',
            'languages': ['English', 'French', 'Kinyarwanda', 'Swahili'],
            'latitude': '30.05944',
            'legislature': 'Parliament (Inteko Ishinga Amategeko / Parlement)',
            'longitude': '-1.94389',
            'mayor': 'Pudence Rubingisa',
            'name': 'Rwanda',
            'political_party': 'Rwandan Patriotic Front',
            'population': 12955768,
            'time_zones': [2.0],
            'url': 'https://en.wikipedia.org/wiki/Rw

           'gdp_nominal': 34895,
           'gdp_pp': 185742,
           'gini': 34.2,
           'hdi': 0.51,
           'head_of_state': 'Abdel Fattah al-Burhan',
           'languages': ['Arabic', 'English'],
           'latitude': '32.56000',
           'legislature': 'National Legislature',
           'longitude': '15.50056',
           'mayor': None,
           'name': 'Sudan',
           'political_party': None,
           'population': 43931980,
           'time_zones': [2.0],
           'url': 'https://en.wikipedia.org/wiki/Sudan',
           'url_head_of_state': 'https://en.wikipedia.org/wiki/Abdel_Fattah_al-Burhan',
           'water_percentage': 6.97},
 'Suriname': {'area': 163820.0,
              'calling_code': '+597',
              'capital_city_area': 182.0,
              'capital_city_name': 'Paramaribo',
              'capital_city_population': 254147,
              'capital_city_url': 'https://en.wikipedia.org/wiki/Paramaribo',
              'continent': 'Americas',


                   'longitude': '38.9101',
                   'mayor': 'Washington DullesReagan '
                            'NationalBaltimore/Washington',
                   'name': 'United States',
                   'political_party': 'Democratic',
                   'population': 332666785,
                   'time_zones': [-12.0,
                                  -11.0,
                                  -10.0,
                                  -9.0,
                                  -8.0,
                                  -7.0,
                                  -6.0,
                                  -5.0,
                                  -4.0,
                                  10.0,
                                  12.0],
                   'url': 'https://en.wikipedia.org/wiki/United_States',
                   'url_head_of_state': 'https://en.wikipedia.org/wiki/Joe_Biden',
                   'water_percentage': 3.96},
 'Uruguay': {'area': 176215.0,
             'calling_cod

# Final Steps

***Note:*** I will still look at COVID data for the bonus in an upcoming section. But I decided to export to CSVs first in order to guarantee the main part of the project.

There are two final steps:
1. Data Cleaning and Final Processing
2. Initially Exporting CSVs
3. Editing CSVs into MySQL-loadable form 

## Data Cleaning

### LANGUAGES OF BOLIVIA

Bolivia has 35+ languages. These were listed in the Country Languages Tables as one entry, so we want to change them correctly.


In [54]:
specialURL = "https://en.wikipedia.org/wiki/Languages_of_Bolivia"
boliviaResponse = requests.get( url=specialURL )
boliviaSoup = BeautifulSoup(boliviaResponse.content, features="html.parser")

wantedAnchor = boliviaSoup.find("a", title="Constitution of Bolivia")
listOfLanguages = wantedAnchor.parent.find_next("ul")

lis = listOfLanguages.find_all("li")

finalList = []
for li in lis:
    if li.text not in finalList: #to prevent duplicates
        finalList.append(li.text)

countriesDict["Bolivia"]["languages"] = finalList

In [55]:
# pprint (countriesDict)

### "AMERICAS" CONTINENT

In [56]:
specialURL = "https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_in_South_America"

amResponse = requests.get( url=specialURL )
amSoup = BeautifulSoup(amResponse.content, features="html.parser")

cells = amSoup.tbody.find_all("td")

for cName, country in countriesDict.items():
    if country["continent"] == "Americas":
        for cell in cells:
            anchor = cell.find("a", title=cName)
            if anchor is not None:
                countriesDict[cName]["continent"] = "South America"

for cName, country in countriesDict.items():
    if country["continent"] == "Americas":
        countriesDict[cName]["continent"] = "North America"



In [57]:
# pprint (countriesDict)

### MAYORS

For some reason, the Mayor of Paris (France) glitched and was set to None, despite everything indictating that it should've worked as normal. If you are curious, you can see the code I have written in the section called "Playground".

The Mayor of London (United Kingdom) also glitched and was set to None, but this time for a known reason. The structure of wikipedia was just too ugly. I did not think it was worth the effort to write a special case to handle this single mayor.

Of course there are other Mayors that were set to None that I did not notice. I noticed these two because they were popular cities. Nevertheless, I made sure that my initial code was a rigorous and as meticulous as I possibly can.

In [58]:
countriesDict["France"]["mayor"] = "Anne Hidalgo"
countriesDict["United Kingdom"]["mayor"] = "Sadiq Khan"

In [59]:
# pprint (countriesDict)

### DATE ASSUMED OFFICE

Date Assumed Office for Countries ruled by Queen Elizabeth II have some wrong values, due to initially using the same day for all countries (which was the day the Queen started ruling UK and other countries, but not all of them).

In [60]:
specialURL = "https://en.wikipedia.org/wiki/List_of_current_state_leaders_by_date_of_assumption_of_office"

daoResponse = requests.get( url=specialURL )
daoSoup = BeautifulSoup(daoResponse.content, features="html.parser")

for cName, country in countriesDict.items():
    if country["head_of_state"] == "Elizabeth II":
        wantedAnchor = daoSoup.find("a", title=cName)
        daoCell = wantedAnchor.find_parent("td").find_next_sibling("td")
        if ":" in daoCell.text:
            tempString = daoCell.text.replace("Queen: ", "")
            tempStringSplit = tempString.split("–", 1)
            date = tempStringSplit[0]
            countriesDict[cName]["date_assumed_office"] = date.strip()

In [61]:
# pprint (countriesDict)

### COORDINATES

Vaitcan City as a city had no coordinates. Coordinates are my PK in this table, so they are a must.

In [62]:
countriesDict["Vatican City"]["longitude"] = 41.904755
countriesDict["Vatican City"]["latitude"] = 12.454628

In [63]:
#pprint (countriesDict)

### CURRENCY

Some currencies were multivalued. Although it would be nice to have currency as a multivalued attribute with its own table, it is not very feasible right now. Thankfully, the project requirements specify only one currency, so I will just use the first currency for each country.

Initially, I thought they would be many, but when I checked with some printing, they turned out to only be three (well, as far as printing can tell me - I'm sure I must've missed a handful). Therefore, I will just hardwire them.

The printing is in the next cell but commented out for readability. Feel free to uncomment it and see for yourself.

In [64]:
# for cName, country in countriesDict.items():
#     if "\n" in country["currency"] or "," in country["currency"] or len(country["currency"]) > 30:
#         print (cName, "\n", country["currency"], "\n")

In [65]:
countriesDict["Eswatini"]["currency"] = "Lilangeni (SZL)"
countriesDict["Tuvalu"]["currency"] = "Tuvaluan dollar"
countriesDict["State of Palestine"]["currency"] = "Israeli new shekel (ILS)" #I researched and this is the most popular of the 3 currencies

In [66]:
# pprint (countriesDict)

### CALLING CODE

Some calling codes were multivalued, because they had different calling codes for different cities / regions. We are only concerned with the main calling code.

I checked the number of countries with multivalued calling codes (with some printing), and they turned out to be a few. Therefore, I will just hardwire them.

The printing is in the next cell but commented out for readability. Feel free to uncomment it and see for yourself.

In [67]:
# for cName, country in countriesDict.items():
#     if country["calling_code"] is not None:
#         if "\n" in country["calling_code"] or "," in country["calling_code"] or len(country["calling_code"]) > 30:
#             print (cName, "\n", country["calling_code"], "\n")

In [68]:
countriesDict["China"]["calling_code"] = "+86"
countriesDict["Denmark"]["calling_code"] = "+45"
countriesDict["Dominican Republic"]["calling_code"] = "+1-809"
countriesDict["Jamaica"]["calling_code"] = "+1-876"
countriesDict["Kazakhstan"]["calling_code"] = "+7-6xx"
countriesDict["Netherlands"]["calling_code"] = "+31"
countriesDict["San Marino"]["calling_code"] = "+378"

In [69]:
# pprint (countriesDict)

### POLITICAL PARTY

Some political parties were incorrect due to the weird structure adopted by Wikipedia, specfically: Wikipedia often lists an old party only, or an old party before current parties. This obviously resulted in some incorrect data. Luckily, I was keeping tabs of all parties by printing them before storing them, so I knew which were faulty. I managed to tweak my code to salvage a lot of them, but the rest had an extremely different structure than the majority of political parties, so I decided to change them later.

There is a total of 15 Heads of States with incorrect political parties.
They can be split into two groups:
1. Heads of State who are currently Independent but have only an old political party (count: 2)
2. Heads of State whose current Political Party is listed after their old parties (count: 3)

The first group can simply be hardwired to the correct value.

In [70]:
countriesDict["Latvia"]["political_party"] = "Independent"
countriesDict["Mauritius"]["political_party"] = "Independent"

The second group can be changed as follows:

In [71]:
buriedPoliticalParty = ["Burkina Faso", "Chile", "Cyprus", "Czech Republic", "Georgia (country)",
                        "Ivory Coast", "Kenya", "Montenegro", "Senegal",
                        "Serbia", "Slovakia", "Slovenia", "Zimbabwe"
                        ]

for c in buriedPoliticalParty:
    bppResponse = requests.get( url = countriesDict[c]["url_head_of_state"])
    bppSoup = BeautifulSoup(bppResponse.content, features="html.parser")
    header = bppSoup.find("th", text = "Political party")
    row = header.find_parent("tr")
    dataCell = row.find("td")
    
    tempString = dataCell.text
    tempStringRightSplit = tempString.rsplit(")", 2)
    tempString = tempStringRightSplit[-2]
    tempStringSplit = tempString.split("]")
    tempString = tempStringSplit[-1] #could be 1 if "]" existed or could be 0 if did not exist. -1 is safe
    tempStringSplit = tempString.split("(", 1)
    
    data = tempStringSplit[0]
    
    countriesDict[c]["political_party"] = data.strip()

In [72]:
# pprint (countriesDict)

### COUNTRY NAMES

Some countries need their names changed, because their names were not very accurate due to scraping them from the "title" attibute of an "a" tag.

Luckily, we were smart enough to save a name attribute in the country's key-value pair. In addition to the CSV benefit, this will also allows us to modify the country's name inside the attribute inside of renaming the key (which is a common Python problem with poor solutions / workarounds).

In [73]:
countriesDict["Georgia (country)"]["name"] = "Georgia"

In [77]:
pprint (countriesDict)

{'Afghanistan': {'area': 652864.0,
                 'calling_code': '+93',
                 'capital_city_area': 275.0,
                 'capital_city_name': 'Kabul',
                 'capital_city_population': 3140853,
                 'capital_city_url': 'https://en.wikipedia.org/wiki/Kabul',
                 'continent': 'Asia',
                 'covid_cases': '156397',
                 'covid_vaccinations': '3398410',
                 'currency': 'Afghani (افغانی) (AFN)',
                 'date_assumed_office': '19 August 2021',
                 'date_of_birth': None,
                 'driving_side': 'R',
                 'gdp_nominal': 17876,
                 'gdp_pp': 83370,
                 'gini': None,
                 'hdi': 0.511,
                 'head_of_state': 'Hibatullah Akhundzada',
                 'languages': ['Pashto', 'Dari'],
                 'latitude': '69.17833',
                 'legislature': 'National Assembly (ملی شورا Mili Shura / '
                      

              'date_of_birth': '1963-06-18',
              'driving_side': 'R',
              'gdp_nominal': 67925,
              'gdp_pp': 174998,
              'gini': 40.0,
              'hdi': 0.816,
              'head_of_state': 'Rumen Radev',
              'languages': ['Bulgarian'],
              'latitude': '23.33',
              'legislature': 'National Assembly (Народно събрание Narodno '
                             'sabranie)',
              'longitude': '42.70',
              'mayor': 'Yordanka Fandakova',
              'name': 'Bulgaria',
              'political_party': 'Independent',
              'population': 6875040,
              'time_zones': [2.0],
              'url': 'https://en.wikipedia.org/wiki/Bulgaria',
              'url_head_of_state': 'https://en.wikipedia.org/wiki/Rumen_Radev',
              'water_percentage': 2.16},
 'Burkina Faso': {'area': 274222.0,
                  'calling_code': '+226',
                  'capital_city_area': 520.0,
            

                'url_head_of_state': 'https://en.wikipedia.org/wiki/Francisco_Guterres',
                'water_percentage': 0.0},
 'Ecuador': {'area': 276841.0,
             'calling_code': '+593',
             'capital_city_area': 372.39,
             'capital_city_name': 'Quito',
             'capital_city_population': 1504991,
             'capital_city_url': 'https://en.wikipedia.org/wiki/Quito',
             'continent': 'South America',
             'covid_cases': '519386',
             'covid_vaccinations': '12686924',
             'currency': 'United States dollarb (USD)',
             'date_assumed_office': '24 May 2021',
             'date_of_birth': '1955-11-16',
             'driving_side': 'R',
             'gdp_nominal': 107435,
             'gdp_pp': 201194,
             'gini': 45.4,
             'hdi': 0.759,
             'head_of_state': 'Guillermo Lasso',
             'languages': ['Spanish', 'Quechua '],
             'latitude': '-78.51250',
             'legislatu

              'capital_city_name': 'Tegucigalpa',
              'capital_city_population': 735982,
              'capital_city_url': 'https://en.wikipedia.org/wiki/Tegucigalpa',
              'continent': 'North America',
              'covid_cases': '376281',
              'covid_vaccinations': '3847285',
              'currency': 'Lempira (HNL)',
              'date_assumed_office': '27 January 2014',
              'date_of_birth': '1968-10-28',
              'driving_side': 'R',
              'gdp_nominal': 25095,
              'gdp_pp': 57649,
              'gini': 52.1,
              'hdi': 0.632,
              'head_of_state': 'Juan Orlando Hernández',
              'languages': ['Spanish'],
              'latitude': '-87.217',
              'legislature': 'National Congress (Congreso Nacional)',
              'longitude': '14.100',
              'mayor': 'Juan García',
              'name': 'Honduras',
              'political_party': 'National Party',
              'population'

                               'députés / Abgeordnetenkammer)',
                'name': 'Luxembourg',
                'political_party': None,
                'population': 634730,
                'time_zones': [1.0],
                'url': 'https://en.wikipedia.org/wiki/Luxembourg',
                'url_head_of_state': 'https://en.wikipedia.org/wiki/Henri,_Grand_Duke_of_Luxembourg',
                'water_percentage': 0.0},
 'Madagascar': {'area': 587041.0,
                'calling_code': '+261',
                'capital_city_area': 88.0,
                'capital_city_name': 'Antananarivo',
                'capital_city_population': 1613375,
                'capital_city_url': 'https://en.wikipedia.org/wiki/Antananarivo',
                'continent': 'Africa',
                'covid_cases': '43632',
                'covid_vaccinations': '381633',
                'currency': 'Ariary (MGA)',
                'date_assumed_office': '19 January 2019',
                'date_of_birth': '1974

                     'water_percentage': 1.09},
 'Norway': {'area': 385207.0,
            'calling_code': '+47',
            'capital_city_area': 480.75,
            'capital_city_name': 'Oslo',
            'capital_city_population': 645701,
            'capital_city_url': 'https://en.wikipedia.org/wiki/Oslo',
            'continent': 'Europe',
            'covid_cases': '216855',
            'covid_vaccinations': '4210216',
            'currency': 'Norwegian krone (NOK)',
            'date_assumed_office': '17 January 1991',
            'date_of_birth': '1937-02-21',
            'driving_side': 'R',
            'gdp_nominal': 403336,
            'gdp_pp': 374994,
            'gini': 25.4,
            'hdi': 0.957,
            'head_of_state': 'Harald V of Norway',
            'languages': ['Norwegian', 'Sami'],
            'latitude': '10.75222',
            'legislature': 'Great Assembly',
            'longitude': '59.91389',
            'mayor': 'Marianne Borgen',
            'name'

             'capital_city_population': 1030594,
             'capital_city_url': 'https://en.wikipedia.org/wiki/Dakar',
             'continent': 'Africa',
             'covid_cases': '73935',
             'covid_vaccinations': '1281042',
             'currency': 'West African CFA franc (XOF)',
             'date_assumed_office': '2 April 2012',
             'date_of_birth': '1961-12-11',
             'driving_side': 'R',
             'gdp_nominal': 23664,
             'gdp_pp': 62412,
             'gini': 40.3,
             'hdi': 0.512,
             'head_of_state': 'Macky Sall',
             'languages': ['French'],
             'latitude': '-17.44667',
             'legislature': 'Parliament',
             'longitude': '14.69278',
             'mayor': 'Soham El Wardini',
             'name': 'Senegal',
             'political_party': 'Alliance for the Republic',
             'population': 17223497,
             'time_zones': [0.0],
             'url': 'https://en.wikipedia.org/wi

 'The Bahamas': {'area': 13943.0,
                 'calling_code': '+1 242',
                 'capital_city_area': 207.0,
                 'capital_city_name': 'Nassau',
                 'capital_city_population': 248948,
                 'capital_city_url': 'https://en.wikipedia.org/wiki/Nassau,_Bahamas',
                 'continent': 'North America',
                 'covid_cases': '22485',
                 'covid_vaccinations': '143726',
                 'currency': 'Bahamian dollar (BSD)',
                 'date_assumed_office': '10 July 1973',
                 'date_of_birth': '1926-04-21',
                 'driving_side': 'L',
                 'gdp_nominal': 13578,
                 'gdp_pp': 13254,
                 'gini': None,
                 'hdi': 0.814,
                 'head_of_state': 'Elizabeth II',
                 'languages': ['English'],
                 'latitude': '-77.333',
                 'legislature': 'Parliament',
                 'longitude': '25.067',
     

                  'covid_cases': '27',
                  'covid_vaccinations': None,
                  'currency': 'Euro (€) (EUR)',
                  'date_assumed_office': '13 March 2013',
                  'date_of_birth': '1936-12-17',
                  'driving_side': 'R',
                  'gdp_nominal': None,
                  'gdp_pp': None,
                  'gini': None,
                  'hdi': None,
                  'head_of_state': 'Pope Francis',
                  'languages': ['Italian'],
                  'latitude': 12.454628,
                  'legislature': 'Pontifical Commission (Pontificia '
                                 'Commissione / Pontificia Commissio)',
                  'longitude': 41.904755,
                  'mayor': None,
                  'name': 'Vatican City',
                  'political_party': None,
                  'population': 825,
                  'time_zones': [1.0],
                  'url': 'https://en.wikipedia.org/wiki/Vatican_City',


## Exporting CSVs

Sources used:
1. Writing to CSV from nested dictionary: 
    https://stackoverflow.com/questions/29400631/python-writing-nested-dictionary-to-csv

2. Getting a sub-dictionary from a subset of keys
    https://stackoverflow.com/questions/5352546/extract-subset-of-key-value-pairs-from-dictionary

### COUNTRY

In [85]:
countryFields = list(countriesDict["Egypt"].keys())

unwantedFields = ['url',
                  'time_zones', 'languages',
                  'url_head_of_state', 'date_of_birth', 'political_party',
                  'capital_city_name', 'capital_city_url', 'capital_city_population', 'capital_city_area',
                  'longitude', 'latitude', 'mayor']

for unwantedField in unwantedFields:
    countryFields.remove(unwantedField)
    
# print(countryFields)

finalCountriesDict = {}
for cName, country in countriesDict.items():
    finalCountriesDict[cName] = {}
    
    for field in list(country.keys()):
        if field in countryFields:
            singleEntry.update({field : country[field]})
    
    if singleEntry["date_assumed_office"] is not None:
        try:
            print (singleEntry["date_assumed_office"])
            if not singleEntry["date_assumed_office"][0].isnumeric():
                singleEntry["date_assumed_office"] = datetime.datetime.strptime(singleEntry["date_assumed_office"].strip(), '%B %d, %Y').strftime('%Y-%m-%d')
            else:
                singleEntry["date_assumed_office"] = datetime.datetime.strptime(singleEntry["date_assumed_office"].strip(), '%d %B %Y').strftime('%Y-%m-%d')
        except:
            print(singleEntry["name"], "had an error in date conversion")
        print (singleEntry["date_assumed_office"])
    
    finalCountriesDict[cName].update(singleEntry)


    
    
with open ("country.csv", "w", newline='', encoding='utf-8') as outfile:
    w = csv.DictWriter( outfile, countryFields )
    w.writeheader()
    for cName,country in sorted(finalCountriesDict.items()):
        try:
            row = {}
            row.update(dict((field, country[field]) for field in (countryFields)))
            w.writerow(row)
        except:
            print (cName, "error")

19 August 2021
2021-08-19
24 July 2017
2017-07-24
19 December 2019
2019-12-19
12 May 2003
2003-05-12
26 September 2017
2017-09-26
1 November 1981
1981-11-01
10 December 2019
2019-12-10
9 April 2018
2018-04-09
6 February 1952
1952-02-06
26 January 2017
2017-01-26
31 October 2003
2003-10-31
10 July 1973
1973-07-10
14 February 2002
2002-02-14
14 March 2013
2013-03-14
30 November 1966
1966-11-30
20 July 1994 
1994-07-20
21 July 2013
2013-07-21
21 September 1981
1981-09-21
6 April 2016
2016-04-06
9 December 2006
2006-12-09
8 November 2020
2020-11-08
1 August 2021
2021-08-01
1 April 2018
2018-04-01
1 January 2019
2019-01-01
5 October 1967
1967-10-05
22 January 2017
2017-01-22
29 December 2015
2015-12-29
18 June 2020
2020-06-18
14 October 2004
2004-10-14
6 November 1982
1982-11-06
6 February 1952
1952-02-06
9 September 2011
2011-09-09
30 March 2016
2016-03-30
20 April 2021
2021-04-20
11 March 2018
2018-03-11
15 November 2012
2012-11-15
7 August 2018
2018-08-07
26 May 2019
2019-05-26
24 Januar

### TIME ZONES

In [86]:
timezoneFields = ["name", "time_zone"]
    
count = 0
timezoneDict = {}
for cName, country in countriesDict.items():
    for tz in country["time_zones"]:
        timezoneDict["#"+str(count)] = {}
        singleEntry = {"name": country["name"], "time_zone" : tz}
        timezoneDict["#"+str(count)].update(singleEntry)
        count += 1

# print(timezoneDict)


with open ("timezone.csv", "w", newline='', encoding='utf-8') as outfile:
    w = csv.DictWriter( outfile, timezoneFields )
    w.writeheader()
    for key,value in sorted(timezoneDict.items()):
        try:
            row = {}
            row.update(dict((field, value[field]) for field in (timezoneFields)))
            w.writerow(row)
        except:
            print (value, "error")

### LANGUAGES

In [87]:
languageFields = ["name", "language"]
    
count = 0
languageDict = {}
for cName, country in countriesDict.items():
    if country["languages"] is not None: 
        for lang in country["languages"]:
            languageDict["#"+str(count)] = {}
            singleEntry = {"name": country["name"], "language" : lang}
            languageDict["#"+str(count)].update(singleEntry)
            count += 1
    else:
        #we shouldn't create an entry at all for a country with no official languages because language is part of the composite PK
        count += 1

# print(languageDict)


with open ("language.csv", "w", newline='', encoding='utf-8') as outfile:
    w = csv.DictWriter( outfile, languageFields )
    w.writeheader()
    for key,value in sorted(languageDict.items()):
        try:
            row = {}
            row.update(dict((field, value[field]) for field in (languageFields)))
            w.writerow(row)
        except:
            print (value, "error")

### CAPITAL CITY

In [ ]:
cityFields = ['name',
              'capital_city_name',
              'capital_city_population', 'capital_city_area',
              'longitude', 'latitude',
              'mayor'
               ]

with open ("city.csv", "w", newline='', encoding='utf-8') as outfile:
    w = csv.DictWriter( outfile, cityFields )
    w.writeheader()
    for cName,country in sorted(countriesDict.items()):
        try:
            row = {}
            row.update(dict((field, country[field]) for field in (cityFields)))
            w.writerow(row)
        except:
            print(cName, "error")

### HEAD OF STATE

In [89]:
headofstateFields = ['head_of_state',
                     'date_of_birth', 'political_party'
                     #we dont need data_assumed_office because we store it in the country
                    ]

headofstateDict = {}
for cName, country in countriesDict.items():
    headofstateDict["#"+str(count)] = {}
    singleEntry = {"head_of_state" : country["head_of_state"],
                   'date_of_birth' : country["date_of_birth"],
                   'political_party' : country["political_party"]
                  }
    if len(singleEntry["date_of_birth"]) == 4: #i.e. if we stored just the year as DOB
        singleEntry["date_of_birth"] = None #this will cause errors in loading to the DB, so just Nullify it
    if singleEntry not in headofstateDict.values(): #avoid duplicates
        headofstateDict["#"+str(count)].update(singleEntry)
        count += 1


with open ("headofstate.csv", "w", newline='', encoding='utf-8') as outfile:
    w = csv.DictWriter( outfile, headofstateFields )
    w.writeheader()
    for cName,country in sorted(headofstateDict.items()):
        row = {}
        row.update(dict((field, country[field]) for field in (headofstateFields)))
        w.writerow(row)

### USERS

In [90]:
userFields = list(userlist[0].keys())
# print (userFields)

with open ("user.csv", "w", newline='', encoding='utf-8') as outfile:
    w = csv.DictWriter( outfile, userFields )
    w.writeheader()
    w.writerows(userlist)

### VISITS

In [91]:
visitFields = list(visits[0].keys())
# print (userFields)

with open ("visit.csv", "w", newline='', encoding='utf-8') as outfile:
    w = csv.DictWriter( outfile, visitFields )
    w.writeheader()
    w.writerows(visits)

## Final CSVs

The CSVs that we have exported above are good, but they will cause some errors when loading them into MySQL. We need to encapsulate the CSVs.

Sources: https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas

In [92]:
df = pd.read_csv('country.csv')
df.dropna(how="all", inplace=True)
df = df.replace(r'^\s*$', np.nan, regex=True)
df.to_csv('country_latin2_replace.csv', encoding='latin2', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC, errors="replace", na_rep="NULL")

In [93]:
df = pd.read_csv('timezone.csv')
df.dropna(how="all", inplace=True)
df = df.replace(r'^\s*$', np.nan, regex=True)
df.to_csv('timezone_latin2_replace.csv', encoding='latin2', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC, errors="replace", na_rep="NULL")

In [94]:
df = pd.read_csv('language.csv')
df.dropna(how="all", inplace=True)
df = df.replace(r'^\s*$', np.nan, regex=True)
df.to_csv('language_latin2_replace.csv', encoding='latin2', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC, errors="replace", na_rep="NULL")

In [95]:
df = pd.read_csv('city.csv')
df.dropna(how="all", inplace=True)
df = df.replace(r'^\s*$', np.nan, regex=True)
df.to_csv('city_latin2_replace.csv', encoding='latin2', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC, errors="replace", na_rep="NULL")

In [96]:
df = pd.read_csv('headofstate.csv')
df.dropna(how="all", inplace=True)
df = df.replace(r'^\s*$', np.nan, regex=True)
df.to_csv('headofstate_latin2_replace.csv', encoding='latin2', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC, errors="replace", na_rep="NULL")

In [97]:
df = pd.read_csv('user.csv')
df.dropna(how="all", inplace=True)
df = df.replace(r'^\s*$', np.nan, regex=True)
df.to_csv('user_latin2_replace.csv', encoding='latin2', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC, errors="replace", na_rep="NULL")

In [98]:
df = pd.read_csv('visit.csv')
df.dropna(how="all", inplace=True)
df = df.replace(r'^\s*$', np.nan, regex=True)
df.to_csv('visit_latin2_replace.csv', encoding='latin2', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC, errors="replace", na_rep="NULL")

# Playground

For testing purposes

In [102]:
#Proof of France

cName = "France"
country = countriesDict[cName]

capitalCityResponse = requests.get( url=country["capital_city_url"] )
capitalCitySoup = BeautifulSoup(capitalCityResponse.content, features="html.parser")
infobox = capitalCitySoup.select_one(".infobox.vcard")


synonyms = [
            "Mayor",
            "Governor",
            "President of the Governorate",
            "Chairman",
            "Major",
            "Thrompon",
            "Maire",
            "General Manager",
            "Intendant",
            "mayor",
            "Council president"
            ]

#I learned this regex search technique from this link:
#https://stackoverflow.com/questions/33406313/how-to-match-any-string-from-a-list-of-strings-in-regular-expressions-in-python
#I used find_all not find because sometimes I get "Governorate" not "Governor"
#I tried changing my regular expression to look for words terminated by end of line or end of string,,
#But I was not able to find an answer within the time constraints
allMatches = infobox.find_all("th", class_="infobox-label")
# print (allMatches)
for match in allMatches:
    if "Mayor" in match.text:
        
        parentRow = match.find_parent("tr")
        if parentRow is not None:
            nameCell = parentRow.td
            if nameCell is not None:
                tempString = nameCell.text
                tempStringSplit = tempString.split("[", 1)
                tempString = tempStringSplit[0]
                tempStringSplit = tempString.split("(", 1)
                print(tempStringSplit[0].strip())
                


Anne Hidalgo


In [103]:
# #Exporting CSVs Trial

# fields = list(countriesDict["Egypt"].keys())
# sampletest = countriesDict

# #https://stackoverflow.com/questions/29400631/python-writing-nested-dictionary-to-csv
# with open ("testttt.csv", "w", newline='', encoding='utf-8') as outfile:
#     w = csv.DictWriter( outfile, fields )
#     w.writeheader()
#     for key,val in sorted(sampletest.items()):
#         try:
#             row = {'name': key}
#             row.update(val)
#             w.writerow(row)
#         except:
#             print (key, "error")

Finally Done. Hope you enjoy the Geopedia.